In [ ]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex
import math

import import_ipynb
import data_utils as data
tt = gp.Model('IIITB Course Timetable')

In [ ]:

# make a dict of sessions --- each course splits into as many sessions are there are
# during the week for that course
# add other course attributes as needed like the enrollment strength etc.
def get_number_of_sessions(course:int):
    return 
# sessions = {}
course_session_map = {}
cnt=0
for course in range(data.n_courses):
    course_session_map[course] = []
    for i in range(get_number_of_sessions(course)):
        course_session_map[course].append(cnt)
        cnt+=1
# n_sessions = len(sessions)
n_sessions = cnt

print(course_session_map)
print(n_sessions)


In [ ]:
def session_length(s:int):
    # get session length (in 15 min chunks) for the course corresponding to the session 's'
    return 6 # used the default value here

session_spans = {s: data.begin_end_pairs(session_length(s)) for s in range(n_sessions)}

S = tt.addVars(
    [
        (b,e,d,s)
        for s in range(n_sessions)
        for d in range(data.n_days)
        for b, e in session_spans[s]
    ],
    vtype=GRB.BINARY,
    name="S",
)

# Each course is assigned a classroom through these indicator variables
C = tt.addVars(
    [
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name="C",
)

## Hard Constraints
##### Each session gets exactly one slot during the week: $\forall s: \sum_{b,e,d} S_{beds}=1$
##### Each Course gets exactly one room: $\forall c: \sum_r C_{cr}=1$
##### Course enrollment cannot exceed room capacity: $\forall c, r: C_{cr}. \texttt{enrollment}(c)\leq \texttt{capacity}(r)$
##### At most one session assigned to any room at any given time & day: $\forall d, t: \sum_{c,r,s,b,e}C_{cr}.S_{beds}\leq  1$
$s$ is a session corresponding to course 'c', (b,e) is a valid time interval for session 's' containing time 't'. Note that $C_{cr}.S_{beds}=1$ if and only if course 'c' is assigned room 'r' and a session 's' for the course is scheduled on day 'd' to start at 'b' and end at 'e'. Clearly there can be at most one such session containing 't'.

In [ ]:
# ----- Each session gets exactly one slot during the week ------------------
tt.addConstrs((S.sum('*','*','*',s) == 1 for s in range(n_sessions)),
              name='one_slot_per_session'
             )

# ----- Each course gets exactly one classroom --------------------
tt.addConstrs((C.sum(c,'*') == 1 for c in range(data.n_courses)),
              name='one_room_per_course'
             )

# ----- Course enrollment does not exceed the classroom capacity --------------------
def capacity(room:int):
    # returns the capacity of the room given the room id (in the 'classrooms' dataframe)
    return 1

def enrollment(c:int):
    # returns the enrollment strength of the course, given the course id (in the 'courses' dataframe)
    return 1

# Capacity constraints
tt.addConstrs((C[c,r]*enrollment(c) <= capacity(r)
              for c in range(data.n_courses)
              for r in range(data.n_rooms)),
              name='course_room_fit'
             )

# -------------- At most one session assigned to any room at any given time ---------------
def classes_for_course(course:int):
    # Return the list of class indices for the given course
    return course_session_map[course]

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in session_spans[s] if not data.overlap(b,e,t)]
tt.update()

In [ ]:

for d in range(data.n_days):
    for t in range(data.n_times):
        tt.addConstr(gp.quicksum(C[c,r]*S[b,e,d,s]
                            for c in range(data.n_courses)
                            for r in range(data.n_rooms)
                            for s in classes_for_course(c)
                            for b,e in covering_sessions(s, t)) <= 1)
tt.update()

## Other Hard Constraints

In [ ]:
# Every Faculty member can be teaching at most one course at a time

for d in range(data.n_days):
    for t in range(data.n_times):
        # every faculty will take only one class
        for f in range(data.n_faculties):
            tt.addConstr(gp.quicksum(S[b, e, d, s]
                                     for c in data.get_courses_by_faculty(f)
                                     for s in classes_for_course(c)
                                     for b, e in covering_sessions(s, t)) <= 1
                                     , name = 'faculty_constraint')

tt.update()


In [ ]:
# core courses for the same batch cannot clash

## Soft Constraints

In [ ]:
# preferred time slots for courses / faculty
# at least one day gap between two consecutive sessions of a course
# one constraint for each student --- to ensure there is no clash
# only lab courses have to be slotted at 2PM on Monday or Wednesday (high priority -- though not a hard constraint)

# add the slack variables to the list of variables

In [ ]:
def is_lab_tutorial(s:int):
    # Returns true if the session with index 's' is a lab/tutorial course
    return False

lab_tut_slack = tt.addVar(vtype=GRB.BINARY, name="lab_tut_slack")
print(lab_tut_slack.getAttr(GRB.Attr.VarName))

tt.addConstrs((S[b,e,d,s] - lab_tut_slack == 0
              for s in range(n_sessions) if not is_lab_tutorial(s)
              for d, t in data.no_lecture_slots
              for b, e in covering_sessions(s, t)
              ),
              name='no_lecture_slots')

## Objective Function

In [ ]:
# Minimize Sum
# --- elective clashes
# --- slack for soft constraints (possibly with weights)

In [ ]:
# weights = {lab_tut_slack.get(gp.): 1}
# objective = gp.quicksum(tt.getVarByName(name)*w for name, w in weights.items())
# tt.setObjective(objective, sense=GRB.MINIMIZE)




## Optimize

In [ ]:
tt.optimize()